In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/03 02:34:47 WARN Utils: Your hostname, codespaces-7c27fa resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/03/03 02:34:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/03 02:34:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Question 1
spark.version

'3.5.1'

In [ ]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-10.csv.gz

In [4]:
!ls -lh fhv_tripdata_2019-10.csv.gz

-rw-rw-rw- 1 codespace codespace 19M Dec  2  2022 fhv_tripdata_2019-10.csv.gz


24/03/03 02:35:02 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [5]:
df = spark.read \
    .option("header", "true") \
    .csv('fhv_tripdata_2019-10.csv.gz')

In [6]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', StringType(), True), StructField('DOlocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [7]:
import pandas as pd

In [8]:
df_pandas = pd.read_csv('fhv_tripdata_2019-10.csv.gz', compression='gzip', nrows=1000)

In [9]:
df_pandas.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID              float64
DOlocationID              float64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [10]:
spark.createDataFrame(df_pandas).schema

/home/codespace/.python/current/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [11]:
schema = types.StructType([
  types.StructField('dispatching_base_num', types.StringType(), True), 
  types.StructField('pickup_datetime', types.TimestampType(), True), 
  types.StructField('dropOff_datetime', types.TimestampType(), True), 
  types.StructField('PUlocationID', types.IntegerType(), True), 
  types.StructField('DOlocationID', types.IntegerType(), True), 
  types.StructField('SR_Flag', types.StringType(), True), 
  types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [12]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv.gz')

In [13]:
df = df.repartition(6)

In [14]:
df.write.parquet('fhv/2019/10/', mode='overwrite')

In [15]:
# Question 2    
!ls -l fhv/2019/10/

total 39052
-rw-r--r-- 1 codespace codespace       0 Mar  3 02:35 _SUCCESS
-rw-r--r-- 1 codespace codespace 6667136 Mar  3 02:35 part-00000-3ee9051e-5644-4144-b9b0-360785cfcd66-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 6649667 Mar  3 02:35 part-00001-3ee9051e-5644-4144-b9b0-360785cfcd66-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 6677707 Mar  3 02:35 part-00002-3ee9051e-5644-4144-b9b0-360785cfcd66-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 6652422 Mar  3 02:35 part-00003-3ee9051e-5644-4144-b9b0-360785cfcd66-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 6666225 Mar  3 02:35 part-00004-3ee9051e-5644-4144-b9b0-360785cfcd66-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 6663171 Mar  3 02:35 part-00005-3ee9051e-5644-4144-b9b0-360785cfcd66-c000.snappy.parquet


In [16]:
df = spark.read.parquet('fhv/2019/10/')

In [17]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [18]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02784|2019-10-01 09:55:38|2019-10-01 10:05:43|          89|          85|   NULL|                  NULL|
|              B02429|2019-10-21 04:15:47|2019-10-21 04:36:04|         264|         264|   NULL|                B02429|
|              B01482|2019-10-19 12:00:00|2019-10-19 12:20:00|         264|         264|   NULL|                B01482|
|              B03015|2019-10-11 14:28:00|2019-10-11 14:32:44|         264|         216|   NULL|                B03015|
|              B01529|2019-10-21 18:00:26|2019-10-21 18:07:21|         264|          80|   NULL|                B01529|
|              B00477|2019-10-03 19:30:3

In [19]:
df.registerTempTable('fhv')

/home/codespace/.python/current/lib/python3.10/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [20]:
df_result = spark.sql("""
SELECT
    pickup_datetime,
    dropOff_datetime, 
    PUlocationID,
    DOlocationID
FROM
    fhv
WHERE
    DATE(pickup_datetime) = '2019-10-15'
""")

In [21]:
df_result.show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|
+-------------------+-------------------+------------+------------+
|2019-10-15 11:02:40|2019-10-15 11:15:33|          15|         252|
|2019-10-15 22:22:47|2019-10-15 22:33:34|         264|          94|
|2019-10-15 10:38:50|2019-10-15 10:59:04|         264|         213|
|2019-10-15 07:37:02|2019-10-15 07:57:07|         264|         212|
|2019-10-15 18:37:00|2019-10-15 19:01:00|         264|         264|
|2019-10-15 13:40:14|2019-10-15 14:45:38|         264|         264|
|2019-10-15 17:36:36|2019-10-15 17:46:25|         264|         127|
|2019-10-15 11:23:33|2019-10-15 11:46:09|         264|         138|
|2019-10-15 10:40:38|2019-10-15 10:55:43|         264|          78|
|2019-10-15 01:30:46|2019-10-15 01:33:48|         264|         227|
|2019-10-15 02:13:36|2019-10-15 02:22:54|         264|         254|
|2019-10-15 13:58:35|2019-10-15 14:17:03|       

In [22]:
# Question 3
df_result.count()

62610

In [23]:
#Question 4
df \
    .withColumn('trip_length', ((df.dropOff_datetime.cast('long') - df.pickup_datetime.cast('long'))/3600)) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .orderBy('trip_length', ascending=False) \
    .limit(5) \
    .show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------------+-----------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|      trip_length|pickup_date|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------------+-----------+
|              B02832|2019-10-11 18:00:00|2091-10-11 18:30:00|         264|         264|   NULL|                B02832|         631152.5| 2019-10-11|
|              B02832|2019-10-28 09:00:00|2091-10-28 09:30:00|         264|         264|   NULL|                B02832|         631152.5| 2019-10-28|
|              B02416|2019-10-31 23:46:33|2029-11-01 00:13:00|        NULL|        NULL|   NULL|                B02416|87672.44083333333| 2019-10-31|
|     B00746         |2019-10-01 21:43:42|2027-10-01 21:45:23|         159|         264|   NULL|    

In [25]:
df_zones = spark.read.parquet('../code/zones/')

In [26]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [27]:
df_result = df.join(df_zones, df.PUlocationID == df_zones.LocationID)

In [28]:
df_result.registerTempTable('fhv_zones')

/home/codespace/.python/current/lib/python3.10/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [29]:
#Question 6
df_result = spark.sql("""
SELECT
    COUNT(1) AS trips,
    Zone
FROM
    fhv_zones
GROUP BY 
    Zone
ORDER BY
    trips ASC
""")

In [30]:
df_result.show()

+-----+--------------------+
|trips|                Zone|
+-----+--------------------+
|    1|         Jamaica Bay|
|    2|Governor's Island...|
|    5| Green-Wood Cemetery|
|    8|       Broad Channel|
|   14|     Highbridge Park|
|   15|        Battery Park|
|   23|Saint Michaels Ce...|
|   25|Breezy Point/Fort...|
|   26|Marine Park/Floyd...|
|   29|        Astoria Park|
|   39|    Inwood Hill Park|
|   47|       Willets Point|
|   53|Forest Park/Highl...|
|   57|  Brooklyn Navy Yard|
|   62|        Crotona Park|
|   77|        Country Club|
|   89|     Freshkills Park|
|   98|       Prospect Park|
|  105|     Columbia Street|
|  110|  South Williamsburg|
+-----+--------------------+
only showing top 20 rows



In [31]:
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x7f503966fd90>>